In [1]:
from mat73 import loadmat
import h5py
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm

In [41]:
Path = 'D:\PulseDB_Vital'
file = 'p000001.mat'

mat_data = loadmat(os.path.join(Path, file))

In [42]:
mat_data['Subj_Wins'].keys()

dict_keys(['ABP_F', 'ABP_Lag', 'ABP_Raw', 'ABP_SPeaks', 'ABP_Turns', 'Age', 'BMI', 'CaseID', 'ECG_F', 'ECG_RPeaks', 'ECG_Raw', 'ECG_Record', 'ECG_Record_F', 'Gender', 'Height', 'IncludeFlag', 'PPG_ABP_Corr', 'PPG_F', 'PPG_Raw', 'PPG_Record', 'PPG_Record_F', 'PPG_SPeaks', 'PPG_Turns', 'SegDBP', 'SegSBP', 'SegmentID', 'SubjectID', 'T', 'Weight', 'WinID', 'WinSeqID'])

In [29]:
print(mat_data['Subj_Wins']['PPG_Raw'][0][0].shape)
mat_data['Subj_Wins']['PPG_Raw'][0][0]

(1250,)


array([0.09463953, 0.10125966, 0.10965594, ..., 0.14367954, 0.11535997,
       0.09382765])

In [43]:
mat_data['Subj_Wins']['SegmentID'][0][0]

array(55.)

In [5]:
#Make a new folder to house the modified mat files
os.mkdir('D:\PulseDB_Vital_modified')

In [31]:
input_file = os.path.join(Path, file)

# Load the .mat file
mat_data = loadmat(input_file)

# Extract CaseID and SegmentID from the .mat file
case_ids_list = mat_data['Subj_Wins']['CaseID']
segment_ids_list = mat_data['Subj_Wins']['SegmentID']

# Convert case_ids_list and segment_ids_list from lists of arrays to flat lists
case_ids = [case_id[0] for case_id in case_ids_list]
segment_ids = [segment_id[0] for segment_id in segment_ids_list]

# Convert to a DataFrame
data = {
    'CaseID': case_ids,
    'SegmentID': segment_ids
}
df = pd.DataFrame(data)

In [39]:
#Checking if any of the files has more than 1 CaseID
for file in tqdm(os.listdir(Path)):
    input_file = os.path.join(Path, file)
    # Extract CaseID and SegmentID from the .mat file
    case_ids_list = mat_data['Subj_Wins']['CaseID']
    segment_ids_list = mat_data['Subj_Wins']['SegmentID']

    # Convert case_ids_list and segment_ids_list from lists of arrays to flat lists
    case_ids = [case_id[0] for case_id in case_ids_list]
    segment_ids = [segment_id[0] for segment_id in segment_ids_list]

    # Convert to a DataFrame
    data = {
        'CaseID': case_ids,
        'SegmentID': segment_ids
    }
    df = pd.DataFrame(data)

    if df.CaseID.value_counts().index.size > 1:
        print(f'File {file} has more than 1 CaseID')
        break

  0%|          | 0/2939 [00:00<?, ?it/s]

In [44]:
print(df.CaseID.value_counts().index.size)
df.CaseID.value_counts()

1


CaseID
c0822    1353
Name: count, dtype: int64

#### Unlike MIMIC_DB, Vital_DB does not contain multiple caseIDs per patient, and the caseID does not encode the time of admission.

In [45]:
# List of keys to include within the 'Subj_Wins' group
keys_to_include = [
    'ABP_Raw', 'Age', 'ECG_Raw', 'Gender',
    'PPG_Raw', 'SegmentID', 'SubjectID', 'T'
]

def process_data(data, output_group):
    for key, value in data.items():
        if key not in keys_to_include:
            continue
        
        if isinstance(value, dict):
            sub_group = output_group.create_group(key)
            process_data(value, sub_group)
        else:
            try:
                if isinstance(value, list):
                    value = np.array(value)
                if isinstance(value, np.ndarray) and value.dtype == 'object':
                    uniform_array = np.array([np.asarray(x) for x in value])
                    output_group.create_dataset(key, data=uniform_array)
                elif isinstance(value, np.ndarray) and value.size == 1:
                    scalar_value = value.item()
                    output_group.create_dataset(key, data=scalar_value)
                elif isinstance(value, np.ndarray) and np.issubdtype(value.dtype, np.str_):
                    output_group.create_dataset(key, data=value.astype('S'))
                else:
                    output_group.create_dataset(key, data=value)
            except Exception as e:
                print(f"Error processing key {key}: {e}")

def process_mat_file(input_file, output_file):
    # Load the .mat file
    mat_data = loadmat(input_file)

    # Extract SegmentID from the .mat file
    # case_ids_list = mat_data['Subj_Wins']['CaseID']
    segment_ids_list = mat_data['Subj_Wins']['SegmentID']

    if type(segment_ids_list) == list:
        # Convert segment_ids_list from lists of arrays to flat lists
        # case_ids = [case_id[0] for case_id in case_ids_list]
        segment_ids = [segment_id[0] for segment_id in segment_ids_list]

        # Convert to a DataFrame
        data = {
            # 'CaseID': case_ids,
            'SegmentID': segment_ids
        }
        df = pd.DataFrame(data)

        # Convert CaseID to datetime
        # df['CaseID_datetime'] = pd.to_datetime(df['CaseID'], format='%Y-%m-%d-%H-%M')

        # Calculate the time of the segment (in seconds)
        df['SegmentTime'] = df['SegmentID'] * 10

        # Add the segment time to CaseID_datetime
        # df['FinalTime'] = df['CaseID_datetime'] + pd.to_timedelta(df['SegmentTime'], unit='s')

        # Convert FinalTime to string format to store back in .mat file
        df['FinalTime_str'] = pd.to_datetime(df['SegmentTime'], unit='s').dt.strftime('%Y-%m-%d-%H-%M-%S')

        Segment_Time = df['FinalTime_str'].values.reshape(-1, 1)

        # Add Segment_Time to the original data
        mat_data['Subj_Wins']['Segment_Time'] = Segment_Time

        # Save the data back into a .h5py file with only the necessary variables
        with h5py.File(output_file, 'w') as out_f:
            subj_wins_group = out_f.create_group('Subj_Wins')
            process_data(mat_data['Subj_Wins'], subj_wins_group)
            subj_wins_group.create_dataset('Segment_Time', data=Segment_Time.astype('S'))

def process_multiple_mat_files(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    mat_files = [file_name for file_name in os.listdir(input_dir) if file_name.endswith('.mat')]
    for file_name in tqdm(mat_files, desc="Processing .mat files"):
        input_file = os.path.join(input_dir, file_name)
        output_file = os.path.join(output_dir, file_name)
        process_mat_file(input_file, output_file)

# Example usage
input_directory = 'D:/PulseDB_Vital'
output_directory = 'D:/PulseDB_Vital_modified'
process_multiple_mat_files(input_directory, output_directory)

Processing .mat files:   0%|          | 0/2938 [00:00<?, ?it/s]

In [66]:
def load_h5_file(file_path):
    def recursively_load_data(h5_obj):
        if isinstance(h5_obj, h5py.Dataset):
            data = h5_obj[()]
            if isinstance(data, bytes):  # Decode byte strings
                return data.decode()
            elif isinstance(data, np.ndarray) and data.dtype.type is np.bytes_:
                return data.astype(str)  # Decode byte strings in numpy arrays
            return data
        elif isinstance(h5_obj, h5py.Group):
            data = {}
            for key, item in h5_obj.items():
                data[key] = recursively_load_data(item)
            return data
        else:
            raise TypeError(f"Unsupported type: {type(h5_obj)}")
    
    # Open the HDF5 file and load data
    with h5py.File(file_path, 'r') as h5_file:
        data = recursively_load_data(h5_file)
    
    return data

In [70]:
input_file = 'D:/PulseDB_Vital/p000003.mat'
input_file_modified = 'D:/PulseDB_Vital_modified/p000003.mat'

unmodified_data = loadmat(input_file)
modified_data = load_h5_file(input_file_modified)

#Compare the keys available in the unmodified and modified data
print(unmodified_data['Subj_Wins'].keys())
print(modified_data['Subj_Wins'].keys())

dict_keys(['ABP_F', 'ABP_Lag', 'ABP_Raw', 'ABP_SPeaks', 'ABP_Turns', 'Age', 'BMI', 'CaseID', 'ECG_F', 'ECG_RPeaks', 'ECG_Raw', 'ECG_Record', 'ECG_Record_F', 'Gender', 'Height', 'IncludeFlag', 'PPG_ABP_Corr', 'PPG_F', 'PPG_Raw', 'PPG_Record', 'PPG_Record_F', 'PPG_SPeaks', 'PPG_Turns', 'SegDBP', 'SegSBP', 'SegmentID', 'SubjectID', 'T', 'Weight', 'WinID', 'WinSeqID'])
dict_keys(['ABP_Raw', 'Age', 'ECG_Raw', 'Gender', 'PPG_Raw', 'SegmentID', 'Segment_Time', 'SubjectID', 'T'])


In [73]:
# Check if the segmentID and Segment_Time correspond to the same time
'''
Remember that segmentID represents a time stamp in seconds divided by 10 
so to change from the time displayed in segmentID to the one in Segment_Time:
    1. Multiply the number in segmentID by 10
    2. Divide by 60 to get the minutes
    3. Multiply the remainder by 60 to get the seconds
'''
print(unmodified_data['Subj_Wins']['SegmentID'][15][0] * 10)
modified_data['Subj_Wins']['Segment_Time'][15][0]

210.0


'1970-01-01-00-03-30'